In [1]:
import os
import pandas as pd
from nltk.metrics.agreement import AnnotationTask
import krippendorff
import numpy as np
from collections import Counter

In [2]:
df_agg_test = pd.read_csv("./clean_collection/step1/s1g1.tsv", sep = '\t')
df_raw_test = pd.read_csv("./clean_collection/step1/s1g1r.tsv", sep = '\t')

In [3]:
df_agg_test.head()

,INPUT:text,OUTPUT:result,CONFIDENCE:result,GOLDEN:result,Unnamed: 4
0,вполне вероятно- поврежден матрикс..нужен врач...,no,99.99%,NaN,NaN
1,Работа проституткой это стыдно? интересует сов...,no,95.89%,NaN,NaN
2,"пожарные, выдвижные-сборные для крыш в основном",no,99.99%,NaN,NaN
3,Помогите решить задачу по нахождению уровня бе...,no,99.99%,NaN,NaN
4,Больше автоматизации уволеных рабочих людей не...,yes,95.27%,NaN,NaN


In [4]:
df_raw_test.dropna(subset = ['OUTPUT:result'], inplace = True)
df_raw_test[['ASSIGNMENT:worker_id','INPUT:text','OUTPUT:result']].head(20)

,ASSIGNMENT:worker_id,INPUT:text,OUTPUT:result
0,b0df6d039a6ba8737dbe14c3abf69935,"тебе доктора говорят, ты - не веришь, а дворни...",yes
1,b0df6d039a6ba8737dbe14c3abf69935,какие бывают комбинации в покере,no
2,b0df6d039a6ba8737dbe14c3abf69935,А как работает карбюратор на движках с турбона...,no
3,b0df6d039a6ba8737dbe14c3abf69935,"Слушай, тут дети, может не будешь такие пошлос...",no
4,b0df6d039a6ba8737dbe14c3abf69935,Сколько зарабатывает начинающий фотограф в час...,yes
5,b0df6d039a6ba8737dbe14c3abf69935,"Вы устраиваетесь, мусорщиком или уборщиком в с...",no
6,b0df6d039a6ba8737dbe14c3abf69935,Как можно наказать если нет преступления?Я кон...,no
7,b0df6d039a6ba8737dbe14c3abf69935,Идёшь в посольство РУ и получаешь паспорт,yes
8,b0df6d039a6ba8737dbe14c3abf69935,Сосиски на костре. Что может быть лучше?Ну еще...,yes
9,b0df6d039a6ba8737dbe14c3abf69935,Можешь. Минимальную пенсию тебе будут платить ...,no


In [49]:
def get_unique_texts_from_df(path):
    df = pd.read_csv(path, sep = '\t')
    df = df[df['GOLDEN:result'].isna()]
    df.dropna(subset = ['INPUT:text'], inplace = True)
    return df, set(df['INPUT:text'])

def calculate_krippendorf_from_raw_df_nltk(df):
    coolected_data = []
    named_collected_data_dict = {}
    
    text2ids = {}
    for t in set(df['INPUT:text'].tolist()):
        text2ids[t] = len(text2ids)
#     print(text2ids)
    df['text_idx'] = df['INPUT:text'].map(text2ids)
    
    worker2id = {}
    for pair in df['ASSIGNMENT:worker_id'].tolist():
        if pair not in worker2id:
            worker2id[pair] = len(worker2id)
    df['worker_idx'] = df['ASSIGNMENT:worker_id'].map(worker2id)
    
    for i,el in df.iterrows():
        coder_idx = el['worker_idx']
        item_idx = el['text_idx']
        label = el['OUTPUT:result']
        
        collected_triplet = (coder_idx,item_idx,label )
        coolected_data.append(collected_triplet)
    
    t = AnnotationTask(data=coolected_data)
    
    return t.alpha(),coolected_data

def krippendorf_with_filtering(collected_data,high_quality_samples_count, constraints_type):
    count_dict = {i:{'no':0,"yes":0} for i in range(high_quality_samples_count)}

    for _ ,item_idx,label in collected_data:
        count_dict[item_idx][label] += 1
        
    value_counts = []
    value_counts_strict = []
    strict_tasks_ids2vote_dict = {}
    for i in range(high_quality_samples_count):
        current_count_element = [0,0]
        for idx, lbl in enumerate(["no","yes"]):
            current_count_element[idx] = count_dict[i][lbl]
     
        value_counts.append(current_count_element)
        
        current_count_element_string = ''.join(sorted([str(el) for el in current_count_element if el != 0]))
        if constraints_type == "top_confidence_only":
            STRICT_CONSTRAINTS = ('2','3','4','5','6','7','8','9','10')
        elif constraints_type in ("top_confidence_vs_oneAND","top_confidence_vs_twoAND"):
             STRICT_CONSTRAINTS = ['2','3','4','5','6','7','8','9','10']
             additional_elements = []
             for strinct_el in STRICT_CONSTRAINTS:
                if strinct_el!= '10':
                    if constraints_type == "top_confidence_vs_oneAND":
                        new_el = '1'+strinct_el
                        additional_elements.append(new_el)
                    elif constraints_type == "top_confidence_vs_twoAND":
                        new_el = '1'+strinct_el
                        additional_elements.append(new_el)
                        
                        new_el = '2'+strinct_el
                        additional_elements.append(new_el)
                        
             STRICT_CONSTRAINTS.extend(additional_elements)
                
        if current_count_element_string in STRICT_CONSTRAINTS:
            value_counts_strict.append(current_count_element)
        
    value_counts_adjusted = [''.join(sorted([str(el) for el in vc if el != 0])) for vc in value_counts]
#     print("MOST COMMON VOTE COMBINATIONS", Counter(value_counts_adjusted).most_common(100))
    
    alpha_no_strict = krippendorff.alpha(value_counts=np.array(value_counts),level_of_measurement='nominal')
    alpha_strict = krippendorff.alpha(value_counts=np.array(value_counts_strict),level_of_measurement='nominal')
    return alpha_no_strict, alpha_strict, value_counts_strict#value_counts_strict можно отсюда сразу формировать датасет с тестом и типом воута

def process_toloka_results_batch(file, agg_confidence_threshold,constraints_type, debug = False):
    coresponding_raw_data_file_path = file[:-4]+'r.tsv'
    
    df_agg, agg_unique_texts = get_unique_texts_from_df(file)
    
    df_raw, raw_unique_texts = get_unique_texts_from_df(coresponding_raw_data_file_path)
    
    if len(raw_unique_texts) != len(agg_unique_texts & raw_unique_texts):
        print(len(raw_unique_texts), len(agg_unique_texts), len(agg_unique_texts & raw_unique_texts))
#     else:
#         print("Unique texts lengths are correct")
       
    df_agg['CONFIDENCE:result'] = df_agg['CONFIDENCE:result'].apply(lambda x: float(x[:-1])/100)
    df_agg_high_quality = df_agg[df_agg['CONFIDENCE:result']>agg_confidence_threshold]
    df_raw_high_quality = df_raw[df_raw['INPUT:text'].isin(df_agg_high_quality['INPUT:text'])]
    
    nltk_alpha,collected_data_alpha = calculate_krippendorf_from_raw_df_nltk(df_raw_high_quality)
    if debug == True:
        print("nltk_alpha", nltk_alpha)
        print("collected_data_alpha",collected_data_alpha[:10])
        print('\n\n')
        
    alpha_no_strict, alpha_strict, value_counts_strict = krippendorf_with_filtering(collected_data_alpha, len(df_agg_high_quality),constraints_type)
    
    assert round(alpha_no_strict,2) == round(nltk_alpha,2), print(alpha_no_strict, nltk_alpha)
    
#     df_raw_report = pd.DataFrame(columns = ['raw_init','raw_init_conf'], 
#                                  data = [[len(df_raw), len(df_raw_high_quality)]])
#     if debug == True: print(df_raw_report) 
    filename = file.split('/')[-1]
    df_agg_report = pd.DataFrame(index = [filename],columns = ['aggreg_samples','aggreg_samples_high_conf',"alpha_high_conf","aggreg_samples_high_conf_and_agreemnt","alpha_high_conf_agreement"], 
                                 data = [[len(df_agg), len(df_agg_high_quality),alpha_no_strict,len(value_counts_strict),alpha_strict]])
    
        
    return df_agg_report, value_counts_strict
    
# process_toloka_results_batch("./clean_collection/step1/s1g1.tsv",0.95,"top_confidence_only")
# process_toloka_results_batch("./clean_collection/step1/s1g1.tsv",0.95,"top_confidence_vs_oneAND")
df_ag_r, vc = process_toloka_results_batch("./clean_collection/step1/s1g1.tsv",0.95,"top_confidence_vs_twoAND")

<ipython-input-49-b49395526492>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_idx'] = df['INPUT:text'].map(text2ids)
<ipython-input-49-b49395526492>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['worker_idx'] = df['ASSIGNMENT:worker_id'].map(worker2id)


In [29]:
import re
# re.match('s\dg\d.tsv','s1g1.tsv')
re.match('s\dg\d.tsv','s1g1t.tsv')

In [50]:
%%capture --no-display

def analysis(confidence_threshold, agreement_filtering_type):
    folder_with_folders = './clean_collection/'
    df_reports_list = []
    value_counts_list = []
    for folder in os.listdir(folder_with_folders):
        final_folder = os.path.join(folder_with_folders, folder)
        if os.path.isdir(final_folder):
            for file in os.listdir(final_folder):
                if re.match('s\dg\d.tsv',file) or re.match('s\d.tsv',file):
                    filepath  = os.path.join(final_folder,file)
                    df_report, value_count = process_toloka_results_batch(filepath,confidence_threshold, agreement_filtering_type)
                    df_reports_list.append(df_report)
                    value_counts_list.extend(value_count)
                    
    df_reports_overall = pd.concat(df_reports_list)
#     alpha_strict_total = krippendorff.alpha(value_counts=np.array(value_counts_list),level_of_measurement='nominal')   
    return df_reports_overall, value_counts_list
dfr, al = analysis(0.95,"top_confidence_vs_twoAND")

In [43]:
len(al)

112093

In [ ]:
%%capture --no-stdout
collected_data = []
for conf in [0.85,0.9,0.95]:
    for aggr_type in ["top_confidence_only","top_confidence_vs_oneAND","top_confidence_vs_twoAND"]:
        df_curr_report , value_counts_filtered = analysis(0.95,"top_confidence_vs_twoAND")
        alpha_strict = krippendorff.alpha(value_counts=np.array(value_counts_filtered),level_of_measurement='nominal')
        number_of_kept_samples_conf = df_curr_report['aggreg_samples_high_conf'].sum()
        number_of_kept_samples_conf_vs_agrement = df_curr_report['aggreg_samples_high_conf_and_agreemnt'].sum()
        
        collected_data.append([conf,number_of_kept_samples_conf, 
                               aggr_type,number_of_kept_samples_conf_vs_agrement, alpha_strict])
df_report = pd.DataFrame(data = collected_data, columns=['confidence','samples_kep_conf_only',
                                                         'strictType','samples_kep_conf_and_aggr','alpha_strict'])
df_report.head(10)

2135 2124 2124
1760 1759 1759
26434 26434 26432
18319 18319 18317
17455 17455 17454
2135 2124 2124
1760 1759 1759
26434 26434 26432
18319 18319 18317
17455 17455 17454
2135 2124 2124
1760 1759 1759
26434 26434 26432
18319 18319 18317
17455 17455 17454
2135 2124 2124
1760 1759 1759
26434 26434 26432
18319 18319 18317
17455 17455 17454
2135 2124 2124
1760 1759 1759
26434 26434 26432
18319 18319 18317
17455 17455 17454
2135 2124 2124
1760 1759 1759
26434 26434 26432
18319 18319 18317
17455 17455 17454
2135 2124 2124
1760 1759 1759
26434 26434 26432
18319 18319 18317
17455 17455 17454
2135 2124 2124
1760 1759 1759
26434 26434 26432
18319 18319 18317
17455 17455 17454
2135 2124 2124
1760 1759 1759
26434 26434 26432
18319 18319 18317


In [44]:
krippendorff.alpha(value_counts=np.array(al),level_of_measurement='nominal')   

0.6843275616654257

In [53]:
dfr['alpha_high_conf_agreement'].mean()

0.6559725303410229

In [52]:
dfr['aggreg_samples_high_conf'].sum()

113804

In [51]:
dfr

,aggreg_samples,aggreg_samples_high_conf,alpha_high_conf,aggreg_samples_high_conf_and_agreemnt,alpha_high_conf_agreement
step1\s1g1.tsv,1017,789,0.461758,699,0.571715
step1\s1g2.tsv,2124,1568,0.563686,1537,0.587646
step1\s1g3.tsv,1759,1494,0.459029,1427,0.508008
step2\s2g1.tsv,6426,5156,0.562238,4932,0.610860
step2\s2g2.tsv,12154,9065,0.619214,8834,0.659446
step2\s2g3.tsv,12053,10081,0.497961,9605,0.546595
step3\s3g1.tsv,18912,14476,0.696933,14235,0.716439
step3\s3g2.tsv,10665,8523,0.675676,8473,0.684468
step3\s3g3.tsv,23975,17748,0.715539,17679,0.721802
step4\s4g1.tsv,26434,15953,0.715383,15876,0.720166
